In [1]:
from importlib import reload as rel
import os, numpy as np, pandas as pd, datetime as dt, time
cwd = os.getcwd()
if cwd.find('src') != len(cwd)-3:
    if cwd.find('algoview') == len(cwd)-8:
        os.chdir(cwd + '/src')
    elif len(cwd) > cwd.find('src')+3:
        os.chdir(cwd[:cwd.find('src')+3])
    else: print('Warning: cannot resolve path')
import queue, collections, trading.utils as utils, trading.functions as fn
import trading.LiveMktData as lmd, trading.OrderPlacement as op, trading.AccountPositions as ap

#### Simulated inputs (from extension)

In [2]:
underlying = 'EURUSD'
msg = 'n=entryL1 d=long t=m p=0 q=1 u=1 c=10000 b=1h'

In [3]:
start_time = time.time()

error = None
info = None
fill_status = False
avg_price = 0
orderid1 = None

try:
    app = fn.reconnect()['app']
    contract_dets = fn.make_contract(app, underlying)
    order1 = fn.make_order(app, contract_dets, msg)

    if order1.totalQuantity != 0:
        orderid1 = app.place_new_IB_order(
            contract_dets['ibcontract'], order1, orderid=None)
        fill_dets = fn.check_fill(app, order1, orderid1)
        fill_status = fill_dets['fill_status']
        avg_price = fill_dets['price']
    else:
        info = "Nothing to trade"
    
    exec_df = fn.get_execDetails(app)
    orders_df = fn.get_openOrders(app)
    
    app.disconnect()
except Exception as Exc:
    app.disconnect()
    error = Exc

end_time = time.time()

IB error id -1 errorcode 2107 string HMDS data farm connection is inactive but should be available upon demand.njhmds.nj
IB error id -1 errorcode 2107 string HMDS data farm connection is inactive but should be available upon demand.njhmds
IB error id -1 errorcode 2106 string HMDS data farm connection is OK:cashhmds
IB error id 50 errorcode 366 string No historical data query found for ticker id:50


In [7]:
app = fn.reconnect()['app']
exec_str = app.get_executions_and_commissions()
app.disconnect()

IB error id -1 errorcode 2107 string HMDS data farm connection is inactive but should be available upon demand.njhmds.nj
IB error id -1 errorcode 2107 string HMDS data farm connection is inactive but should be available upon demand.njhmds
IB error id -1 errorcode 2106 string HMDS data farm connection is OK:cashhmds


In [11]:
exec_list = [str(v) for k, v in exec_str.items()]

In [14]:
exec_str['0000e215.5c3bd9ae.01.01']

Execution - contract: 12087792,EUR,CASH,,0.0,,,IDEALPRO,,USD,EUR.USD,EUR.USD,False,,combo: ClientId: 1 OrderId: 59 time: 20190114  13:11:30 AvgPrice: 1.14696 Price: 1.14696 AcctNumber: DU486001 Shares: 82576.0 Commission: 2.0 commission_currency: USD realisedpnl: 1.7976931348623157e+308

In [12]:
exec_list

['Execution - contract: 12087792,EUR,CASH,,0.0,,,IDEALPRO,,USD,EUR.USD,EUR.USD,False,,combo: ClientId: 1 OrderId: 59 time: 20190114  13:11:30 AvgPrice: 1.14696 Price: 1.14696 AcctNumber: DU486001 Shares: 82576.0 Commission: 2.0 commission_currency: USD realisedpnl: 1.7976931348623157e+308',
 'Execution - contract: 12087792,EUR,CASH,,0.0,,,IDEALPRO,,USD,EUR.USD,EUR.USD,False,,combo: ClientId: 1 OrderId: 60 time: 20190114  13:11:42 AvgPrice: 1.14696 Price: 1.14696 AcctNumber: DU486001 Shares: 82406.0 Commission: 2.0 commission_currency: USD realisedpnl: 1.7976931348623157e+308',
 'Execution - contract: 12087792,EUR,CASH,,0.0,,,IDEALPRO,,USD,EUR.USD,EUR.USD,False,,combo: ClientId: 1 OrderId: 61 time: 20190114  13:28:14 AvgPrice: 1.14619 Price: 1.14619 AcctNumber: DU486001 Shares: 81070.0 Commission: 2.0 commission_currency: USD realisedpnl: 1.7976931348623157e+308',
 'Execution - contract: 12087792,EUR,CASH,,0.0,,,IDEALPRO,,USD,EUR.USD,EUR.USD,False,,combo: ClientId: 1 OrderId: 62 time: 2

In [4]:
exec_df.tail()

,underlying,OrderId,AvgPrice,Shares,conId,ClientId
time,,,,,,
20190114 13:30:15,EURUSD,63,1.14619,81070,12087792,1
20190114 13:31:16,EURUSD,64,1.14614,81070,12087792,1
20190114 13:32:17,EURUSD,65,1.14631,81070,12087792,1
20190114 13:33:18,EURUSD,66,1.14618,81070,12087792,1
20190114 13:39:04,EURUSD,67,1.14603,81070,12087792,1


In [5]:
orders_df.tail()

,status,underlying,direction,order type,quantity,price,filled,avg fill px,conId,ClientId


In [18]:
app = fn.reconnect()['app']
app.cancel_all_orders()
app.disconnect()

IB error id -1 errorcode 2106 string HMDS data farm connection is OK:njhmds.nj
IB error id -1 errorcode 2107 string HMDS data farm connection is inactive but should be available upon demand.njhmds
IB error id -1 errorcode 2106 string HMDS data farm connection is OK:cashhmds
IB error id 59 errorcode 161 string Cancel attempted when order is not in a cancellable state.  Order permId =821852069
IB error id 58 errorcode 202 string Order Canceled - reason:


#### Historical Market Data

In [ ]:
app = fn.reconnect()
ibcontract, minTick = fn.make_contract(app, underlying)
hist_mkt_data = app.get_IB_historical_data(ibcontract, durationStr='2 D', barSizeSetting='1 hour')
app.disconnect()

#### Account Positions

In [ ]:
app = fn.reconnect()

## lets get positions
positions_list = app.get_current_positions()
print(positions_list)

## get the account name from the position
## normally you would know your account name
accountName = positions_list[0][0] #'DU486001'

## and accounting information
accounting_values = app.get_accounting_values(accountName)
print(accounting_values)

## these values are cached
## if we ask again in more than 5 minutes it will update everything
accounting_updates = app.get_accounting_updates(accountName)
print(accounting_updates)

app.disconnect()